# Módulo 2: Carga de Datos - Archivos CSV con Polars

Los archivos CSV (Comma-Separated Values) son un formato estándar para el intercambio de datos tabulares. Polars ofrece una función `pl.read_csv()` altamente optimizada y flexible para leer estos archivos de manera eficiente.

En este notebook, exploraremos las funcionalidades más comunes de `pl.read_csv()`.

In [ ]:
import polars as pl
from io import StringIO # Para simular archivos en memoria

print(f"Polars version: {pl.__version__}")

## CSV Básico con Encabezado

La forma más simple de leer un CSV es proporcionar la ruta al archivo (o un buffer, como `StringIO` para este ejemplo). Polars intentará inferir el separador, si tiene encabezado y los tipos de datos automáticamente.

In [ ]:
csv_data_con_header = "col_a,col_b,col_c\n1,a,true\n2,b,false\n3,c,true"

# Usamos StringIO para simular un archivo en memoria
df_con_header = pl.read_csv(StringIO(csv_data_con_header))

print("DataFrame leído desde CSV con encabezado:")
print(df_con_header)

print("\nGlimpse del DataFrame:")
df_con_header.glimpse()

## Especificar el Separador (`separator`)

Aunque Polars es bueno infiriendo el separador, a veces es necesario especificarlo explícitamente, especialmente si el archivo usa un delimitador no estándar como punto y coma (`;`) o tabulaciones (`\t`).

In [ ]:
csv_data_punto_y_coma = "col_x;col_y;col_z\n10;x;1.1\n20;y;2.2\n30;z;3.3"

df_punto_y_coma = pl.read_csv(StringIO(csv_data_punto_y_coma), separator=';')

print("DataFrame leído desde CSV con separador punto y coma:")
print(df_punto_y_coma)

## CSV Sin Encabezado (`has_header=False`)

Si el archivo CSV no contiene una fila de encabezado, Polars, por defecto, tratará la primera línea como encabezado. Para evitar esto, se usa `has_header=False`. En este caso, Polars asignará nombres de columna genéricos como `column_1`, `column_2`, etc.

Podemos proporcionar nuestros propios nombres de columna usando el parámetro `new_columns`.

In [ ]:
csv_data_sin_header = "100,d,false\n200,e,false\n300,f,true"

# Leer CSV sin encabezado, Polars asigna nombres por defecto
df_sin_header = pl.read_csv(StringIO(csv_data_sin_header), has_header=False)
print("DataFrame sin encabezado (nombres por defecto):")
print(df_sin_header)

# Leer CSV sin encabezado y asignar nombres personalizados
df_sin_header_custom_names = pl.read_csv(
    StringIO(csv_data_sin_header), 
    has_header=False, 
    new_columns=["id", "letra", "estado"]
)
print("\nDataFrame sin encabezado (nombres personalizados):")
print(df_sin_header_custom_names)

## Seleccionar Columnas Específicas (`columns`)

Para optimizar la carga, especialmente con archivos CSV muy anchos, podemos seleccionar solo un subconjunto de columnas. Esto se puede hacer proporcionando una lista de nombres de columna o una lista de índices de columna (0-based) al parámetro `columns`.

In [ ]:
# Reutilizamos csv_data_con_header: "col_a,col_b,col_c\n1,a,true\n2,b,false\n3,c,true"

# Seleccionar columnas por nombre
df_cols_seleccionadas_nombre = pl.read_csv(StringIO(csv_data_con_header), columns=["col_a", "col_c"])
print("Selección de columnas por nombre (['col_a', 'col_c']):")
print(df_cols_seleccionadas_nombre)

# Seleccionar columnas por índice (0-based)
df_cols_seleccionadas_indice = pl.read_csv(StringIO(csv_data_con_header), columns=[0, 2]) # col_a y col_c
print("\nSelección de columnas por índice ([0, 2]):")
print(df_cols_seleccionadas_indice)

## Limitar Número de Filas (`n_rows`) y Omitir Filas (`skip_rows`)

- `n_rows`: Útil para leer solo las primeras `N` filas de un archivo, ideal para inspecciones rápidas de archivos grandes sin cargar todo en memoria.
- `skip_rows`: Permite omitir un número específico de filas desde el inicio del archivo. Esto es útil si el archivo CSV contiene metadatos o líneas de comentarios al principio que no son parte de los datos tabulares.

In [ ]:
# Reutilizamos csv_data_con_header: "col_a,col_b,col_c\n1,a,true\n2,b,false\n3,c,true"

# Leer solo las primeras 2 filas
df_n_rows = pl.read_csv(StringIO(csv_data_con_header), n_rows=2)
print("Leyendo solo las primeras 2 filas (n_rows=2):")
print(df_n_rows)

# Simular un CSV con líneas de metadatos al inicio
csv_data_con_skip = "metadata_line_1\nmetadata_line_2\ncol_1,col_2\nval1,val2\nval3,val4"

# Omitir las primeras 2 filas (las líneas de metadatos)
df_skip_rows = pl.read_csv(StringIO(csv_data_con_skip), skip_rows=2)
print("\nOmitiendo las 2 primeras filas (skip_rows=2):")
print(df_skip_rows)

## Tipos de Datos (`dtypes` o `schema`)

Especificar los tipos de datos es crucial para la correctitud de los datos y la eficiencia de la memoria y el procesamiento. Aunque Polars tiene una inferencia de tipos robusta, hay casos donde es preferible definir los tipos explícitamente.

- `dtypes`: Se puede pasar un diccionario donde las claves son nombres de columna y los valores son los tipos de Polars deseados (e.g., `pl.Int32`, `pl.Float64`, `pl.Utf8`, `pl.Boolean`, `pl.Date`). Las columnas no especificadas en el diccionario seguirán la inferencia automática.
- `schema`: Similar a `dtypes`, pero puede ser una lista de tipos si se conocen todas las columnas en orden, o un diccionario. Es más flexible si se quiere renombrar columnas y especificar tipos al mismo tiempo.

In [ ]:
# Reutilizamos csv_data_con_header: "col_a,col_b,col_c\n1,a,true\n2,b,false\n3,c,true"
# Por defecto, Polars infiere: col_a: Int64, col_b: Utf8, col_c: Boolean
print("Tipos inferidos por defecto:")
pl.read_csv(StringIO(csv_data_con_header)).glimpse()

# Forzar col_a a Float32 y col_c a Utf8 (string)
custom_dtypes = {
    "col_a": pl.Float32, 
    "col_c": pl.Utf8
}

df_custom_dtypes = pl.read_csv(StringIO(csv_data_con_header), dtypes=custom_dtypes)

print("\nTipos con dtypes personalizados:")
df_custom_dtypes.glimpse()

## Manejo de Valores Nulos (`null_values`)

Por defecto, Polars interpreta las cadenas vacías (`""`) como cadenas vacías literales, no como valores nulos. Si tu CSV usa cadenas específicas para representar datos faltantes (e.g., `"NA"`, `"missing"`, `"-"`), puedes indicárselo a Polars mediante el parámetro `null_values`.

`null_values` puede ser:
- Una cadena: `null_values="NA"` (ese string se interpretará como nulo en todas las columnas).
- Una lista de cadenas: `null_values=["NA", "N/A", "-"]` (cualquiera de estos strings se interpretará como nulo).
- Un diccionario: `null_values={"columna_X": "missing", "columna_Y": ""}` (define valores nulos específicos por columna).

In [ ]:
csv_data_con_nulos = "id,valor,desc\n1,10,ok\n2,NA,error\n3,,ok\n4,30,missing"

# Comportamiento por defecto: "NA" y "missing" son strings, "" es string vacío
df_nulos_defecto = pl.read_csv(StringIO(csv_data_con_nulos))
print("Nulos por defecto (conteo de nulos por columna):")
print(df_nulos_defecto.null_count())
print(df_nulos_defecto) # Observa los valores

# Especificar "NA" como valor nulo para todas las columnas
df_nulos_na = pl.read_csv(StringIO(csv_data_con_nulos), null_values="NA")
print("\nNulos con null_values='NA':")
print(df_nulos_na.is_null()) # Muestra una máscara booleana de nulos
print(df_nulos_na.null_count())

# Especificar una lista de valores nulos para todas las columnas
df_nulos_multi = pl.read_csv(StringIO(csv_data_con_nulos), null_values=["NA", "missing", ""])
print("\nNulos con null_values=['NA', 'missing', '']:")
print(df_nulos_multi.is_null())
print(df_nulos_multi.null_count())

## Resumen y Próximos Pasos

Hemos cubierto los parámetros más importantes de `pl.read_csv()` para cargar y parsear archivos CSV de manera efectiva. Polars ofrece muchos más parámetros para situaciones más complejas, como:
- `encoding`: Para especificar la codificación del archivo (e.g., `'utf-8'`, `'latin1'`, `'iso-8859-1'`).
- `comment_char`: Para especificar un carácter que indica que el resto de la línea es un comentario y debe ser ignorado.
- `low_memory`: Reduce el uso de memoria a costa de, potencialmente, leer el archivo en múltiples chunks (puede afectar el rendimiento y la inferencia de tipos si los chunks son muy diferentes).
- `ignore_errors`: Si es `True`, Polars intentará omitir las filas que no se puedan parsear correctamente. Por defecto es `False`, lo que significa que un error de parseo detendrá la operación.
- `try_parse_dates`: Intenta parsear automáticamente columnas que parezcan fechas/horas.

En los siguientes apartados y notebooks, exploraremos cómo guardar DataFrames y cómo trabajar con otros formatos de archivo como Parquet y Excel.